# IBM RXN for Chemistry on COVID19 candidates

An example usage of `rxn4chemistry` using the COVID19 candidates recently released [here](https://www.diamond.ac.uk/covid-19/for-scientists/Main-protease-structure-and-XChem.html) and hosted on Diamond Light Source [website](https://www.diamond.ac.uk/covid-19.html).
In the following we will use `rxn4chemistry` for automated retrosynthesis.

## Get the data

In [ ]:
!curl https://www.diamond.ac.uk/dam/jcr:db16b9c7-fff4-4cc1-bb40-d233c566ff31/Mpro%20full%20XChem%20screen%20-%20experiment%20summary%20-%20ver-2020-03-25-annotated.xlsx -o data.xlsx
!ls *xlsx  

## Parse the data

In [ ]:
import pandas as pd
data = pd.read_excel('data.xlsx').dropna(axis=0)
code_to_smiles = {
    code: smiles
    for code, smiles in zip(data['CompoundCode'], data['CompoundSMILES'])
}

## Instanciate the wrapper

Setup the wrapper using a valida API key. You can get one on the IBM RXN website from [here](https://rxn.res.ibm.com/rxn/user/profile).

In [ ]:
from rxn4chemistry import RXN4ChemistryWrapper
api_key = 'API_KEY'
rxn4chemistry_wrapper = RXN4ChemistryWrapper(api_key=api_key)

## Create a project

Create a project, you can easily check the identifier associated to it in the response.

In [ ]:
response = rxn4chemistry_wrapper.create_project('diamond_light_source_covid19_candidates')

You can get your project id from the wrapper

In [ ]:
rxn4chemistry_wrapper.project_id

## Run  a retrosynthesis using the wrapper

Running a retrosynthesis is as easy as picking a molecule and calling a one-liner.

In [ ]:
compound = 'Z271004858'
smiles = code_to_smiles[compound]

In [ ]:
response = rxn4chemistry_wrapper.predict_automatic_retrosynthesis(product=smiles)

### Check the status and get the results

In [ ]:
# rerun this until the status is 'SUCCESS', keep in mind the server allows only 5 requests per minute
# and a timeout between consecutive requests of 2 seconds
results = rxn4chemistry_wrapper.get_predict_automatic_retrosynthesis_results(response['prediction_id'])
results['status']

### Inspect the predicted paths (upon 'SUCCESS')

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from IPython.display import display


def collect_reactions(tree):
    reactions = []
    if 'children' in tree and len(tree['children']):
        reactions.append(
            AllChem.ReactionFromSmarts('{}>>{}'.format(
                '.'.join([node['smiles'] for node in tree['children']]),
                tree['smiles']
            ), useSmiles=True)
        )
    for node in tree['children']:
        reactions.extend(collect_reactions(node))
    return reactions

In [ ]:
for index, path in enumerate(results['retrosynthetic_paths']):
    print('Showing path {} with confidence {}:'.format(index, path['confidence']))
    for reaction in collect_reactions(path):
        display(Chem.Draw.ReactionToImage(reaction))